In [ ]:
def avg_waiting_time(tasks_durations):
    total_durations = [full_duration(tasks_durations, i) for i in range(len(tasks_durations))]
    avg = sum(total_durations) / len(total_durations)
    return avg


def full_duration(tasks_durations, index):
    previous_durations = tasks_durations[:index]
    current_duration = tasks_durations[index]
    return sum(previous_durations) + current_duration

print(avg_waiting_time([10, 5, 11, 3]))
print(avg_waiting_time([3, 11, 5, 10]))
print(avg_waiting_time([11, 10, 5, 3]))
print(avg_waiting_time([3, 5, 10, 11]))

print("Worst scenario:", avg_waiting_time(sorted([11, 10, 5, 3], reverse=True)))
print("Best scenario:", avg_waiting_time(sorted([11, 10, 5, 3])))


In [ ]:
weighted_tasks = [(10, 20), (5, 20), (11, 50), (3, 10)]

def rate(task):
    duration, priority = task
    return priority / duration

def prioritize_tasks(tasks):
    rated_tasks = [(task[0], task[1], rate(task)) for task in tasks]
    prioritized_tasks = sorted(rated_tasks, key=lambda t: t[2], reverse=True)
    return prioritized_tasks

print(prioritize_tasks(weighted_tasks))


In [ ]:
import heapq
from collections import defaultdict

class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq
    
    def __str__(self):
        return f"{self.char}({self.freq})"

def build_huffman_tree(text):
    # Count character frequencies
    frequency = defaultdict(int)
    for char in text:
        frequency[char] += 1
    
    # Create a priority queue (min-heap)
    heap = [Node(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)
    
    # Build the Huffman tree
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        internal = Node(None, left.freq + right.freq)
        internal.left = left
        internal.right = right
        heapq.heappush(heap, internal)
    
    return heap[0]  # Return the root of the tree

def generate_codes(root, current_code="", codes=None):
    if root is None:
        return
    
    if codes is None:
        codes = {}
    
    if root.char is not None:
        codes[root.char] = current_code
        return
    
    generate_codes(root.left, current_code + "0", codes)
    generate_codes(root.right, current_code + "1", codes)
    return codes


In [ ]:
text = "ABRACADABRA"

root = build_huffman_tree(text)
print(root)

codes = generate_codes(root)
print(codes)

coded = ""
for char in text:
    coded += codes[char]

print(text)
print(coded)

In [ ]:
def compress(text, codes):
    compressed = []
    for char in text:
        compressed.extend(codes[char])
    
    compressed_bits = ''.join(compressed)
    extra_padding = 8 - len(compressed_bits) % 8
    compressed_bits += '0' * extra_padding

    print(compressed_bits)
    
    compressed_bytearray = bytearray()
    for i in range(0, len(compressed_bits), 8):
        byte = compressed_bits[i:i + 8]
        compressed_bytearray.append(int(byte, 2))
    
    return compressed_bytearray, extra_padding

def decompress(compressed, codes, extra_padding):
    compressed_bits = ''.join(f'{byte:08b}' for byte in compressed)
    compressed_bits = compressed_bits[:-extra_padding]
    
    reversed_codes = {code: char for char, code in codes.items()}
    result = []
    current_code = ''
    for bit in compressed_bits:
        current_code += bit
        if current_code in reversed_codes:
            result.append(reversed_codes[current_code])
            current_code = ''
    return ''.join(result)


In [ ]:
text = "abracadabra simsalabim"

root = build_huffman_tree(text)
codes = generate_codes(root)
print(f"Huffman Codes: {codes}")

compressed, extra_padding = compress(text, codes)
print(f"Compressed text (in bytes): {list(compressed)}, {extra_padding}")

decompressed = decompress(compressed, codes, extra_padding)
print(f"Decompressed text: {decompressed}")


original_size = len(text) * 8
compressed_size = len(compressed) * 8 - extra_padding
print(f"Original size: {original_size} bit")
print(f"Size after compression: {compressed_size} bit")
print(f"Compression ratio: {original_size / compressed_size:.2f}")
